In [ ]:
# @IMPORT-MERGE
from plaster.run.run import RunResult
from plaster.run.plots import plots
from plaster.tools.ipynb_helpers import displays
from munch import Munch
from plumbum import local
from IPython.display import HTML
import numpy as np
import pandas as pd
from plaster.tools.zplots import zplots
z = zplots.setup()

In [ ]:
# @REMOVE-FROM-TEMPLATE
from plumbum import local
print(local.cwd)

src = local.path( "../../../jobs_folder/jhm_20200124_tfawashout_methanol_4min_220frames/sigproc_0_4_minute"  )
run = RunResult( src )
displays.title( src.name )


# Signal Visualization ("movie" format)

In [ ]:
displays.subtitle("Overview")
plots.text_sigproc_overview(run)

In [ ]:
displays.subtitle( "Quality")
plots.plot_sigproc_stats(run)

In [ ]:
# Histograms of signal and signal/noise (SNR) for first and last cycles
#
titles = ['Peak signal distribution by field, first/last cycles',
          'Peak SNR distribution by field, first/last cycles' ]
all_titles = [ "Peak signal distribution, all fields combined, first/last cycle",
               "Peak SNR distribution, all fields combined, first/last cycle"]
div_noise = [ False, True ]

for title,all_title,is_snr in zip( titles, all_titles, div_noise ):
    # Do a first pass to get range so all plots can be same scale
    max_x = 0
    max_y = 0
    for field in range(run.sigproc.n_fields):
        _mx,_my = plots.plot_channel_signal_histograms( run, limit_field=field, limit_cycles=[0,run.sigproc.n_cycles-1], div_noise=is_snr, _range_only=True )
        max_x = max( _mx, max_x )
        max_y = max( _my, max_y )

    # And then plot.  If this is single-channel data, which is tools in photobleaching experiments, 
    # lay this out horizontally.
    displays.subtitle(title)
    if run.sigproc.n_channels==1:
        with z(_cols=run.sigproc.n_fields):
            for field in range(run.sigproc.n_fields):
                plots.plot_channel_signal_histograms( run, limit_field=field, limit_cycles=[0], div_noise=is_snr, f_x_range=(0,max_x), f_y_range=(0,max_y*1.1), _cols=run.sigproc.n_fields, _zplot_context=z )
            for field in range(run.sigproc.n_fields):
                plots.plot_channel_signal_histograms( run, limit_field=field, limit_cycles=[run.sigproc.n_cycles-1], div_noise=is_snr, f_x_range=(0,max_x), f_y_range=(0,max_y*1.1), _cols=run.sigproc.n_fields, _zplot_context=z )
    else:
        for field in range(run.sigproc.n_fields):
            plots.plot_channel_signal_histograms( run, limit_field=field, limit_cycles=[0,run.sigproc.n_cycles-1], div_noise=is_snr, f_x_range=(0,max_x), f_y_range=(0,max_y*1.1) )
    
#     displays.subtitle(all_title)
#     _,_ = plots.plot_channel_signal_histograms( run, limit_cycles=[0,run.sigproc.n_cycles-1], div_noise=is_snr, f_x_range=(0,max_x)  )


# Uncomment to see all frames
#displays.subtitle("Peaks intensity distribution histograms for every cycle, all fields combined")
#plot_channel_signal_histograms( run )


In [ ]:
# photobleaching curve - what is the average signal at each cycle?
df = run.sigproc.fields__n_peaks__peaks__radmat()
displays.subtitle( "Average signal is expected to decay by cycle due to bleaching")
avg_sig = df.groupby('cycle_i')['signal'].mean()
med_sig = df.groupby('cycle_i')['signal'].median()
max_y = max( np.max(avg_sig), np.max(med_sig ) )
with z( _cols=2, f_y_range=[0,max_y*1.1] ):
    z.scat(x=range(avg_sig.shape[0]),y=avg_sig,f_title='average signal by cycle, all fields combined',f_y_axis_label='average signal',f_x_axis_label='cycle')
    z.scat(x=range(med_sig.shape[0]),y=med_sig,f_title='median signal by cycle, all fields combined',f_y_axis_label='median signal',f_x_axis_label='cycle')

In [ ]:
displays.subtitle( "Average signal, signal delta, and signal-delta as percentage, by field")

fields = sorted(df.field_i.unique())
avg_sig = df.groupby(['field_i','cycle_i'])['signal'].mean()

def good_field(f):
    # This is not the case when for some field there is no signal
    return type(avg_sig[f]) == pd.core.series.Series

# compute signal delta
zeros = np.zeros(run.sigproc.n_cycles)
avg_sig_diff = np.array([ np.diff(avg_sig[f]) if good_field(f) else zeros for f in fields ])
avg_sig_diff_pct = np.array([ avg_sig_diff[f] / avg_sig[f][1:] * 100 if good_field(f) else zeros for f in fields ])

# compute the plot ranges to keep them all on same scale
avg_sig_y = ( avg_sig.min() * .9, avg_sig.max() * 1.1)
m = max( abs(avg_sig_diff.min()), abs(avg_sig_diff.max()) ) * 1.1
avg_diff_y = ( -m, m )
m = max( abs(avg_sig_diff_pct.min()), abs(avg_sig_diff_pct.max()) ) * 1.1
avg_diff_pct_y = ( -m, m )

# plot
with z( _cols=3 ):
    for f in fields:
        if good_field(f):
            z.scat( x=range(run.sigproc.n_cycles), y=avg_sig[f], f_title=f'average signal by cycle, field {f}',f_y_axis_label='average signal',f_x_axis_label='cycle',f_y_range=avg_sig_y )
            z.scat( x=range(run.sigproc.n_cycles-1), y=avg_sig_diff[f], f_title=f'average signal delta at cycle, field {f}',f_y_axis_label='signal delta',f_x_axis_label='cycle',f_y_range=avg_diff_y )
            z.scat( x=range(run.sigproc.n_cycles-1), y=avg_sig_diff_pct[f], f_title=f'average signal delta percentage at cycle, field {f}',f_y_axis_label='signal delta percentage',f_x_axis_label='cycle',f_y_range=avg_diff_pct_y )
        else:
            print( f"(No data for field {f})")

# Jag/Jim exploratory functionality for photobleaching analysis

In [ ]:
#
# 1. Early brightness value
#
def get_signal_stats_for_cycles( df, cycle_start, n_cycles=1, channel=0, fields=None ):
    '''
        df          : the sigproc_df from a run
        cycle_start : (int) starting cycle to collect stats for
        n_cycles    : (int) how many cycles to collect status for
        channel     : (int) which channel to include
        fields      : None, int, or list of ints, which field(s) to include
        
        Returns:
            a tuple of brightness stats: (mean, median, stddev)
    '''
    s = df[df['cycle_i'].between(cycle_start,cycle_start+n_cycles-1,inclusive=True)]
    s = s[s.channel_i == channel]
    fs = s.field_i.unique()
    if fields is not None:
        if not isinstance(fields,list):
            fields=[fields]
        s = s[s.field_i.isin(fields)]
           
    mean = s.signal.mean()
    median = s.signal.median()
    std = s.signal.std()
    
    snr = s.signal / s.noise
    snr_mean = snr.mean()
    snr_median = snr.median()
    snr_std  = snr.std()
        
    return (mean,median,std,snr_mean,snr_median,snr_std)
    
#
# Compute stats for brightness values starting at a given cycle, across n_cycles, for a given channel and field(s)
#
cycle_start = 0
n_cycles    = 1
channel     = 0
fields      = None # None means use all fields.  Otherwise give a number or list of numbers, e.g. [0,1,2]
    
mean,median,stddev,snr_mean,snr_median,snr_std = get_signal_stats_for_cycles( run.sigproc.fields__n_peaks__peaks__radmat(), cycle_start=cycle_start, n_cycles=n_cycles, channel=channel, fields=fields )
print( f"mean {mean:g}, median {median:g}, stddev {stddev:g}, snr_mean={snr_mean:g}, snr_median{snr_median:g} snr_std {snr_std:g}" )


## Peak lifetime analysis

In [ ]:
displays.subtitle( "Peak lifetime (intensity-threshold-based), by field")

# What is a reasonable threshold for "dye is ON" to pick automatically?
# If we do some per-field analysis and pick some threshold based on the median, mean, etc
# then this is somewhat circular with respect to talking about half-lives, thresholds, etc.
# If we believe that in general we are running enough cycles such that fluorophores should be
# mostly all bleached by the end, we could look at the average signal for last frame(s)
# and set our threshold above this.  
#
# Really we need to know something about the dye - mean brightness + stddev to set this 
# threshold with confidence.
# compute signal delta

# Get the signal stats for computing threshold from each field, channel 0, last cycle
channel = 0
cycle_start = run.sigproc.n_cycles - 1
n_cycles = 1
radmat = run.sigproc.fields__n_peaks__peaks__radmat()
fields = sorted(radmat.field_i.unique())
n_std = 1 # how many standard-deviations above last-frame mean signal counts as "On"

field_df = run.sigproc.fields()

# Or a user may specify a "hardcoded" threshold to use here:
# (The n_std above mean per field will be used otherwise)
user_threshold = None

for f in fields:
    r = radmat[ radmat.field_i == f ].copy()
    mean0,median0,stddev0,_,_,_ = get_signal_stats_for_cycles( r, cycle_start=0, n_cycles=1, channel=channel )
    mean,median,stddev,_,_,_ = get_signal_stats_for_cycles( r, cycle_start=cycle_start, n_cycles=n_cycles, channel=channel )

    threshold = mean + stddev * n_std  # mean + n_std
    threshold = median
    if user_threshold is not None:
        threshold = user_threshold
    
    r['peak_is_on'] = r.signal > threshold
    g = r.groupby(['peak_i'])
    peak_lifetime = g.apply(lambda grp: run.sigproc.n_cycles if np.all(grp.peak_is_on) else np.where(grp.peak_is_on==False)[0][0] )
    peaks_on_at_cycle =  [ (c <= peak_lifetime.values).sum() for c in range(run.sigproc.n_cycles+1) ]
    
    exposure = field_df[field_df.field_i==f].exposure_time[0]
    
    with z(_cols=4):
        z.scat( x=range(run.sigproc.n_cycles+1), y=peaks_on_at_cycle, f_title=f"Field {f} peaks on at cycle, threshold={int(threshold)}", f_x_axis_label=f"n_cycles ({int(exposure)}ms exposure)", f_y_axis_label="count")
        z.hist( peak_lifetime, _bins=20, f_title=f"Field {f} peak lifetime, threshold={int(threshold)}", f_x_axis_label=f"n_cycles ({int(exposure)}ms exposure)", f_y_axis_label="count")
        

        signals = radmat[(radmat.field_i==f) & (radmat.cycle_i==0)].signal
        ymax = np.max(signals) 
        with z(_merge=True, f_title=f'Field {f} peaks, cycle 0', f_x_axis_label='peak number', f_y_axis_label='signal', f_y_range=[0,ymax]  ):
            z.scat( x=range(len(signals)), y=signals ) 
            z.line( x=range(len(signals)), y=[threshold]*len(signals), color='red', line_width=2 ) 

        signals = radmat[(radmat.field_i==f) & (radmat.cycle_i==run.sigproc.n_cycles-1)].signal
        with z(_merge=True, f_title=f'Field {f} peaks, cycle {run.sigproc.n_cycles-1}', f_x_axis_label='peak number', f_y_axis_label='signal', f_y_range=[0,ymax] ):
            z.scat( x=range(len(signals)), y=signals)  
            z.line( x=range(len(signals)), y=[threshold]*len(signals), color='red', line_width=2 ) 
            
    print ( f'field {f} cycle   0:  mean {int(mean0)}  median {int(median0)} std {int(stddev0)}')
    print ( f'field {f} cycle {cycle_start}:  mean {int(mean)}  median {int(median)} std {int(stddev)}')


In [ ]:
displays.subtitle( "Peak lifetime (SNR-threshold-based), by field")

# What if we pick a threshold for SNR instead of just signal?

# Get the signal stats for computing threshold from each field, channel 0, last cycle
channel = 0
cycle_start = run.sigproc.n_cycles - 1
n_cycles = 1
radmat = run.sigproc.fields__n_peaks__peaks__radmat()
fields = sorted(radmat.field_i.unique())
n_std = 0

field_df = run.sigproc.fields()

# Or a user may specify a "hardcoded" threshold to use here:
# (The n_std above mean per field will be used otherwise)
user_snr_threshold = None

for f in fields:
    r = radmat[ radmat.field_i == f ].copy()
    _,_,_,mean0,median0,stddev0 = get_signal_stats_for_cycles( r, cycle_start=0, n_cycles=1, channel=channel )
    _,_,_,mean,median,stddev = get_signal_stats_for_cycles( r, cycle_start=cycle_start, n_cycles=n_cycles, channel=channel )

    threshold = mean + stddev * n_std  # mean + n_std
    threshold = median
    if user_threshold is not None:
        threshold = user_threshold
    
    r['peak_is_on'] = (r.signal/r.noise) > threshold
    g = r.groupby(['peak_i'])
    peak_lifetime = g.apply(lambda grp: run.sigproc.n_cycles if np.all(grp.peak_is_on) else np.where(grp.peak_is_on==False)[0][0] )
    
    exposure = field_df[field_df.field_i==f].exposure_time[0]
    
    with z(_cols=3):
        z.hist( peak_lifetime, _bins=20, f_title=f"Field {f} peak lifetime, threshold={threshold:.2f}", f_x_axis_label=f"n_cycles ({int(exposure)}ms exposure)", f_y_axis_label="count")

        r = radmat[(radmat.field_i==f) & (radmat.cycle_i==0)]
        signals = r.signal / r.noise
        ymax = np.max(signals) 
        with z(_merge=True, f_title=f'Field {f} peaks, cycle 0', f_x_axis_label='peak number', f_y_axis_label='signal/noise', f_y_range=[0,ymax]  ):
            z.scat( x=range(len(signals)), y=signals ) 
            z.line( x=range(len(signals)), y=[threshold]*len(signals), color='red', line_width=2 ) 

        r = radmat[(radmat.field_i==f) & (radmat.cycle_i==run.sigproc.n_cycles-1)]
        signals = r.signal / r.noise
        with z(_merge=True, f_title=f'Field {f} peaks, cycle {run.sigproc.n_cycles-1}', f_x_axis_label='peak number', f_y_axis_label='signal/noise', f_y_range=[0,ymax] ):
            z.scat( x=range(len(signals)), y=signals)  
            z.line( x=range(len(signals)), y=[threshold]*len(signals), color='red', line_width=2 ) 
            
    print ( f'field {f} cycle   0: SNR mean {mean0:.2f}  median {median0:.2f} std {stddev0:.2f}')
    print ( f'field {f} cycle {cycle_start}: SNR mean {mean:.2f}  median {median:.2f} std {stddev:.2f}')


In [ ]:
# @REMOVE-FROM-TEMPLATE
# e.g. sanity check the number of peaks whose lifetime is 0 because they're under the threshold
# in the first cycle...
field = 0
cycle = 0
signals = radmat[(radmat.field_i==field)&(radmat.cycle_i==cycle)]
len(signals[signals.signal<=4027])



## Half life analysis

In [ ]:
#
# 2. Half life
#
# First some function's we'll call.  Scroll down for user-edited code.
#
from scipy.optimize import curve_fit

def average_signal_all_cycles( df, channel=0, fields=None ):
    '''
        df          : the sigproc_df from a run
        channel     : (int) which channel to include
        fields      : None, int, or list of ints, which field(s) to include
        
        Returns:
            a vector of length n_cycles containing average signal at each cycle
    '''
    s = df[df.channel_i==channel]
    if fields is not None:
        if isinstance(fields,int):
            fields=[fields]
        s = s[s.field_i.isin(fields)]
    avg_sig = s.groupby('cycle_i')['signal'].mean()
    return np.array(avg_sig)

def get_signal_xy( run, channel=0, fields=None ):
    x = np.arange( run.sigproc.n_cycles )
    y = average_signal_all_cycles( run.sigproc.fields__n_peaks__peaks__radmat(), channel=channel, fields=fields )
    return x, y

def single_exp( t, a1, b1, c ):
    return a1 * np.exp(-b1*t) + c

def double_exp( t, a1, b1, a2, b2, c ):
    return a1 * np.exp(-b1*t) + a2 * np.exp(-b2*t) + c

exp1_param_names = ['a1','b1','c']
exp2_param_names = ['a1','b1', 'a2', 'b2', 'c']

def guess_exp_params( x_values, y_values ):
    '''
    The difficulty with all fitting functions is that they often need
    reasonable starting points to get anywhere!
    
    x_values, y_values: the data you hope to fit
    
    Returns:
        p1,p2 - each vectors which are guesses for the exp1 and exp2 
                functions to fit this data.            
    '''    

    six_halflives = 6.0 * np.log(2)
    duration = x_values[-1] - x_values[0]
    y0 = y_values[0]
    yinf = y_values[-1]

    # single-exp guesses
    p1 = [ y0-yinf, six_halflives/duration, yinf ]
    
    # double-exp guesses
    dt1 = duration // 6
    dt2 = duration
    ymid = y_values[dt1]
    b1=six_halflives / dt1
    b2=six_halflives / dt2
    p2 = [y0-ymid,b1,ymid-yinf,b2,yinf]
    
    return p1,p2

def wrapped_curve_fit( fitFn, x, y, p0, data_name='' ):
    '''
    Fit fitFn to data in x,y start at param vector p0.
    Returns param vector, covariance matrix, or None if fit failed
    for some reason (usually failure to converge)

    data_name is optional  and use for error reporting in the case
    of failed fits.
    '''
    try:
        p,c = curve_fit( fitFn, x, y, p0=p0)
    except:
        print( f"{data_name} {fitFn.__name__} fit failed (probably failed to converge)")
        p = c = None
    return p,c

# Put this into an easy form to get half-lives for signal for a given
# field and channel for a run based on some exponential fit.
#
def get_halflife( run, field, channel, n_exponentials ):
    '''
    fit an exponential function with n_exponentials to data in field,channel, 
    and compute the halflife(s).
    Returns:
        a vector of the half-lifes (one for each exponential)
    '''

    assert 0 < n_exponentials < 3
    fitFn = [None,single_exp, double_exp ][n_exponentials]
    
    x,y = get_signal_xy( run, channel=channel, fields=field )
    p = None
    
    if len(y) >= 5:
        exp1_guess_p, exp2_guess_p = guess_exp_params( x, y )
        p0 = [None,exp1_guess_p, exp2_guess_p][n_exponentials]

        p,c = wrapped_curve_fit( fitFn, x, y, p0, f"field{field} ch{ch}" )

    if p is None:
        # If fit failed for some reason
        return [np.nan] * n_exponentials

    # Half-life is function of rate param b1,b2, etc:
    # p = a1,b1,c (single exp) or a1,b1,a2,b2,c (double exp) so --> bn = p[n*2 - 1]
    
    half_lives = []
    for n in range(1,n_exponentials+1):
        rate = p[n*2-1]
        half_lives += [ np.log(2) / rate ]
        
    return half_lives
    
    

##########################################################################################
#
# a. get data you want to fit - the decay of average signal
# 
which_fields = 0  # None for all, or some int or list of fields
which_channel = 0   # which channel to get signal for

fields_label = "All" if which_fields is None else f"{which_fields}"
data_name = f"field:{fields_label} ch:{which_channel}"

test_x_values,test_y_values = get_signal_xy( run, which_channel, which_fields )

if len(test_y_values) < 5: # need at least 5 datapoints for double exp fit.
    print( f"No data to fit in field {which_fields} channel {which_channel}")

else:

    # b. fit it to a couple of functions.  Note to fit you often have to make
    # reasonable guesses at the params, so if you know those (approximately),
    # you can supply them here.  Sometimes the guesses aren't close enough 
    # and the fitter does not converge.  
    #
    exp1_guess_p, exp2_guess_p = guess_exp_params( test_x_values, test_y_values )

    p_exp1,p_cov1 = wrapped_curve_fit( single_exp, test_x_values, test_y_values, exp1_guess_p, data_name )
    p_exp2,p_cov2 = wrapped_curve_fit( double_exp, test_x_values, test_y_values, exp2_guess_p, data_name )

    # c. evaluate functions at optimal params, print fitted params, and plot
    #
    if p_exp1 is not None:
        exp1_eval = [ single_exp( t, p_exp1[0], p_exp1[1], p_exp1[2]) for t in test_x_values ]
        print( '\nSingle Exponential')
        for name,value in zip( exp1_param_names, p_exp1):
            print( f'{name:2s} {value}')
        print(f"\nerr = {np.sqrt(np.diag(p_cov1))}")
        print(f"sse = {np.linalg.norm(test_y_values-exp1_eval)}")

        with z( _merge=True, f_title=f'Signal, single exp: ch={which_channel} field={fields_label}',f_y_axis_label='average signal',f_x_axis_label='cycle' ):
            z.line(x=test_x_values,y=exp1_eval,line_color='red',line_width=2)
            z.scat(x=test_x_values,y=test_y_values,)

    if p_exp2 is not None:
        exp2_eval = [ double_exp( t, p_exp2[0], p_exp2[1], p_exp2[2], p_exp2[3], p_exp2[4]) for t in test_x_values ]
        print( '\nDouble Exponential')
        for name,value in zip( exp2_param_names, p_exp2 ):
            print( f'{name:2s} {value}')
        print(f"\nerr = {np.sqrt(np.diag(p_cov2))}")
        print(f"sse = {np.linalg.norm(test_y_values-exp2_eval)}")

        with z( _merge=True, f_title=f'Signal, double exp: ch={which_channel} field={fields_label}',f_y_axis_label='average signal',f_x_axis_label='cycle' ):
            z.line(x=test_x_values,y=exp2_eval,line_color='red',line_width=2)
            z.scat(x=test_x_values,y=test_y_values,)



## CSV Exports

In [ ]:
#
# 3. Flat file for radiometry matrix ("radmat")
#   
# Grab the peak_i, channel, cycle, signal, noise, and field columns from the sigproc_df
# and emit as a csv.  Sort by channel, peak_i, cycle to get all of the same peak contiguous.

rad_export = run.sigproc.fields__n_peaks__peaks__radmat()[['channel_i','peak_i','cycle_i','field_i','signal','noise']].sort_values(['field_i','channel_i','peak_i','cycle_i'])
filename = f'./report_peaks_{run.run_folder.basename}.csv'
rad_export.to_csv( filename, index=False, float_format="%g" )

displays.md( f'## This table exported as {filename[2:]}')
display( rad_export.head(5) )


In [ ]:
#
# 4. Tabular output per nd2 file (per "field" for "--movie" timeseries captures)
#
# * File name, number_frames, time_per_frame (probably a setting present in the nd2?), number of spots identified,
#   mean_early_brightness (1st frame), median_early_brightness (1st frame), stdev_early_brightness (1st frame), 
# . half_life_1exp (ln2/b; from single exponential fit), half_life_b1 (ln 2/b1), 
# . half_life_b2 (ln2/b2) [these two from two exponential fits]  

# TODO: does the RunResult load manifest files?
# assert run.nd2_import_movie.n_fields == len(run.nd2.manifest.nd2_paths)

col_titles = [ 
    "filename", "n_frames", "exposure_time", "n_peaks", "frame1_signal_mean",
    "frame1_signal_median", "frame1_signal_stddev", "frames1t5_signal_mean",
    "frames1t5_signal_median", "frames1t5_signal_stddev", "half_1exp", "half_2exp_b1", "half_2exp_b2" ]


s = run.sigproc.fields__n_peaks__peaks__radmat()
field_meta = run.ims_import_movie.metadata_by_field()
field_df = run.sigproc.fields()
rows = []
for field_i in range(run.ims_import_movie.n_fields):
    meta = field_meta[field_meta.field_i == field_i]
    df = field_df[(field_df.cycle_i==0)&(field_df.field_i==field_i)]
    for ch in range(run.sigproc.n_channels):
        row = Munch().fromkeys(col_titles)
        row.filename = meta.filename.iloc[0]
        row.n_frames = meta.n_frames.iloc[0]
        row.exposure_time = df.exposure_time.iloc[0]
        row.n_peaks = s[(s.cycle_i==0) & (s.field_i==field_i) & (s.channel_i==ch)].count().peak_i

        mean,median,stddev,_,_,_ = get_signal_stats_for_cycles( s, cycle_start=0, n_cycles=1, channel=ch, fields=field_i )
        row.frame1_signal_mean = mean
        row.frame1_signal_median = median
        row.frame1_signal_stddev = stddev

        mean,median,stddev,_,_,_ = get_signal_stats_for_cycles( s, cycle_start=0, n_cycles=5, channel=ch, fields=field_i )
        row.frames1t5_signal_mean = mean
        row.frames1t5_signal_median = median
        row.frames1t5_signal_stddev = stddev

        row.half_1exp = get_halflife( run, field_i, ch, n_exponentials=1 )[0]
        
        halfs = get_halflife( run, field_i, ch, n_exponentials=2 )
        row.half_2exp_b1 = halfs[0]
        row.half_2exp_b2 = halfs[1]
        
        rows += [row]

summary_df = pd.DataFrame( rows )
filename = f'./report_nd2_{run.run_folder.basename}.csv'
summary_export_df = summary_df[ col_titles ]  # force order of columns
summary_export_df.to_csv( filename, index=False, float_format="%g", na_rep="NaN" )

displays.md( f'## This table is exported as {filename[2:]}')
display(summary_export_df)
    

# Signal: Interactive Wizards

In [ ]:
plots.wizard_scat_df(run, default_x='cycle_i')

In [ ]:
plots.wizard_xy_df(run)

In [ ]:
percentile = 0.50
max_bright = run.sigproc.fields__n_peaks__peaks__radmat().signal.quantile(percentile)
stride = run.sigproc.n_cycles // 10 # display approx 10 frames from sequence
plots.wizard_raw_images(run, show_circles=False, peak_i_square=True, square_radius=7, max_bright=max_bright, cycle_stride=stride)